In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# Remote Connections and SSH

Whether we need to connect to a Hadoop cluster where we're conducting analysis, a web server hosting an app we've developed, or simply our desktop at home while we're at the office, we usually want to protect our local and remote computers and the traffic between them. Secure Shell (SSH) is a protocol for making secure network connections between computers over an insecure network (e.g. the internet).

In this notebook we'll review how use SSH to gain shell access to a remote machine and how to transfer files over SSH. This notebook will assume a Unix-like environment using the OpenSSH client. Other SSH clients are available (including the popular Windows SSH client, PuTTY), but will not be discussed here.

## How SSH Works


Traffic between two computers can easily be encrypted using [symmetric-key encryption](https://en.wikipedia.org/wiki/Symmetric-key_algorithm). However, while this secures the traffic between the two computers, the connection between client and server is only truly secure if both parties to authenticate their identity to the other. This is done using [public-key encryption](https://en.wikipedia.org/wiki/Public-key_cryptography).

For instance, the server generates a pair of cryptographic keys, one private and one public. When the client tries to connect to the server, the server sends the client a message signed with the server's private key. The client keeps a record of public keys from servers it has connected to. The client uses this server's public key to verify that the signature is authentic. Importantly, while the public key can be used to verify the signature, it cannot be used to compute what the private key is. This authenticates that the server is what the client believes, and that the client's request to connect isn't being intercepted by an imposter.

The client can authenticate its identity to the server in one of two ways: via a password or using public-key encryption. Password authentication simply involves the client sending the appropriate password for the user account it's requesting to access on the server. The public-key encryption process is very similar to the public-key encryption process the server uses to authenticate its identity to the client. The client generates a pair of cryptographic keys, one private and one public. The server administrator must add the public key to a record of authorized keys allowed to connect to the server. The client then sends the server a message signed with the client's private key. The server can then use the client's public key to verify the signature.

Before any authentication takes place, all traffic between client and server is encrypted with a symmetric-key algorithm established at the beginning of the process. This protects the authentication process from eavesdropping. Encrypting the authentication process is particularly important for password authentication, since an eavesdropper could easily intercept the password. It is less important for public-key encryption, since the private key is never transmitted and cannot be determined from the signed messages or the public key used for authentication.

## Password Authentication


The most basic way to establish an SSH connection is by logging into a user account on the remote computer using the account's password. The format used is `ssh username@ipaddress`. If it is your first time connecting to the server, SSH will ask if you want to add it to your known hosts. The server will prompt you to enter the password for the user account you are trying to access. Once successfully logged in your terminal will display a shell prompt for the user on the remote machine.

## Keyed Authentication


Passwords are vulnerable to brute force attack, theft, and forgetfulness. It is preferable to authenticate your identity to the server the same way that the server authenticates its identity to you: via a pair of public and private keys.

First we'll generate a pair of keys using `ssh-keygen -t rsa`. You will be prompted for a file in which to store the private key (the public key will be stored at the same path but with `.pub` appended to the file name) and whether you want to protect the private key with a password.

Next you distribute your public key to the server administrator (e.g. the sysadmin at your company or an account management tool on a cloud service provider), who will add it to keys authorized to access the server. Authorized keys are stored in a file of the user account being accessed, typically at `/home/username/.ssh/authorized_keys`.

Finally we can connect to the server by using SSH with the "identity file" option: `ssh -i /path/to/private_key username@ipaddress`.

## SSH Configuration


Usually we will SSH to a small number of computers many times. Rather than trying to remember the IP addresses of these computers and typing them out every time we want to connect, we can give the computers names and tell SSH where to find the private key we use to connect to that computer. Many SSH options can be preset in a configuration file located at `~/.ssh/config`. For instance adding

```bash
Host digitalocean
HostName ipaddress
User jovyan
IdentityFile /path/to/private_key
```

to this config file would enable us to connect to `jovyan@ipaddress` by simply entering `ssh digitalocean`.

## Sending Files Over SSH


SSH grants shell access, but there is no way to direct the shell we've logged into on the remote machine to copy files to our local machine. We can use a related Bash command, `scp`, rather than `ssh` to move files over the connection. The syntax of `scp` is the same as `cp`, except we specify remote paths starting with the address of the remote machine. If we've set up our config file to use the alias `digitalocean` for our remote machine, we could copy the datacourse directory to the home directory of our local machine by executing `scp -r digitalocean:~/datacourse ~`.

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*